In [269]:
import numpy as np
import random
from keras.models import Sequential
from keras.layers import LSTM, Softmax, Dense, UpSampling1D, Conv1D, MaxPooling1D

Lets start out by opening up our poems file and doing some preprocessing to clean it up.

In [277]:
poems = list()

with open("data/shakespeare.txt") as f:
    # The poem we are currently on
    index = 0

    # whether we are starting a new poem based on a line
    new = False
    for line in f:
        # Remove leading spaces and the terminal new-line character
        l = line.lstrip().lower()
        
        # If our line is not blank (blanks mean we are in between poems)
        if(len(list(filter(None, l.split(" "))))>0):
           
            # Check if we already started a new poem
            if(new):
                # Add the line to the current poem
                poems[-1]+=l
                
            # If we have not started, a new poem, start a new poem
            else:
                poems.append('')
                new = True

        # If we are on a blank, move to the next poem and indicate that we have not started a new one
        else:
            index += 1
            new = False

print(poems[0], end='')
print('hi')

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou contracted to thine own bright eyes,
feed'st thy light's flame with self-substantial fuel,
making a famine where abundance lies,
thy self thy foe, to thy sweet self too cruel:
thou that art now the world's fresh ornament,
and only herald to the gaudy spring,
within thine own bud buriest thy content,
and tender churl mak'st waste in niggarding:
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.
hi


In [168]:
inputs = list()
input_size = 40

for poem in poems:
    i = input_size
    while i < len(poem)-1:
        inputs.append([poem[i-input_size:i], poem[i]])
        i+=1

In [169]:
print(inputs[0])
print(inputs[1])
print(inputs[2])

['from fairest creatures we desire increas', 'e']
['rom fairest creatures we desire increase', ',']
['om fairest creatures we desire increase,', '\n']


In [170]:
len(inputs)

87359

In [171]:
encoding = dict()
uniques = list(set(''.join(poems)))

for i, s in enumerate(uniques):
    encoding[s] = i

for k, v in encoding.items():
    print(k, v)

encoding_size = len(encoding)
print(encoding_size)

o 0
e 1
. 2
j 3
, 4
v 5
z 6
w 7
) 8
( 9
p 10
? 11
t 12
f 13
l 14

 15
c 16
u 17
a 18
: 19
! 20
b 21
n 22
' 23
r 24
y 25
d 26
  27
x 28
m 29
q 30
k 31
h 32
- 33
g 34
s 35
; 36
i 37
38


In [172]:
def encode(s):
    out = np.zeros([len(encoding)])
    out[encoding[s]] = 1
    return out

def batch_encode(line):
    #out = np.zeros([encoding_size*len(line)])
    #for i in range(len(line)):
    #    out[(i*encoding_size)+encoding[line[i]]] = 1
    out = list()
    for i in line:
        out.append(encode(i))
    return out 

In [173]:
X = list()
y = list()

for i in inputs:
    X.append(batch_encode(i[0]))
    y.append(encode(i[1]))

X = np.array(X)
y = np.array(y)

print(X)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [174]:
# define model
model = Sequential()

model.add(LSTM(100, input_shape=(X[0].shape)))
model.add(Dense(encoding_size, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dense_5 (Dense)              (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None


In [175]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [203]:
# fit model
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
87359/87359 [==============================] - 113s 1ms/step - loss: 1.3872 - acc: 0.5694
Epoch 2/10
87359/87359 [==============================] - 113s 1ms/step - loss: 1.3750 - acc: 0.5733
Epoch 3/10
87359/87359 [==============================] - 113s 1ms/step - loss: 1.3624 - acc: 0.5754
Epoch 4/10
87359/87359 [==============================] - 113s 1ms/step - loss: 1.3508 - acc: 0.5788
Epoch 5/10
87359/87359 [==============================] - 115s 1ms/step - loss: 1.3395 - acc: 0.5822
Epoch 6/10
87359/87359 [==============================] - 114s 1ms/step - loss: 1.3278 - acc: 0.5854
Epoch 7/10
87359/87359 [==============================] - 115s 1ms/step - loss: 1.3171 - acc: 0.5885
Epoch 8/10
87359/87359 [==============================] - 115s 1ms/step - loss: 1.3068 - acc: 0.5917
Epoch 9/10
87359/87359 [==============================] - 116s 1ms/step - loss: 1.2970 - acc: 0.5940
Epoch 10/10
87359/87359 [==============================] - 118s 1ms/step - loss: 1.2874 - a

Now that we have trained our first model, lets write some functions to generate poems. We need a function which probabilistically selects an item from an array of percents, a function which generates the next character based on an input and a function which generates a poem given a model and a seed string.

In [214]:
def prob(ar):
    i = 0
    r = random.random()
    while(np.sum(ar[0][:i+1])<r):
        i+=1
    return i

In [247]:
def get_next(model, line):
    return uniques[prob(model.predict(np.array(batch_encode(line)).reshape(1, len(line), encoding_size)))]

def get_next_det(model, line):
    return uniques[np.argmax(model.predict(np.array(batch_encode(line)).reshape(1, len(line), encoding_size)))]

In [226]:
output = 'from fairest creatures we desire increas'
get_next(model, output)

'e'

In [245]:
def gen_poem(model, seed='from fairest creatures we desire increas'):
    out = seed
    l = 0
    while l < 14:
        new = get_next(model, out[-len(seed):])
        if(new == '\n'):
            l+=1
        out+=new
    print(out)
    return out

gen_poem(model)

def gen_det_poem(model, seed='from fairest creatures we desire increas'):
    out = seed
    l = 0
    while l < 14:
        new = get_next_det(model, out[-len(seed):])
        if(new == '\n'):
            l+=1
        out+=new
    print(out)
    return out

gen_det_poem(model)

from fairest creatures we desire increast.
but siuguch things truthting thee were thine,
when i live they shall sprindams my dead.
lities volder tond, wist the pition, utward.
their eprent of thes, but thee hamp's poor,
the purpust thee good in buind unjubledd:
my smused, and this geipl can thus a prease.
love that upon his is gives this shim:
how which is you mow, which must assures from love bud,
which he stoe wond ride to heart which it not knows,
whith when those by and carrectares are now,
while is before of the frefound, i dwelled,
is sequent that i quinds so days in dryst.
the mory not i may a hisefowned cond.

from fairest creatures we desire increase,
which i desprice of the time that thou shoughtsome doth state the sun a see thee,
and so thou art thou art is i not to age,
to the time that thou art to the store dead,
that i have such a too and the world with those,
and thou art to the store to the world,
then thand me to the sun a see to my see,
so the sun a self all tooked th

"from fairest creatures we desire increase,\nwhich i desprice of the time that thou shoughtsome doth state the sun a see thee,\nand so thou art thou art is i not to age,\nto the time that thou art to the store dead,\nthat i have such a too and the world with those,\nand thou art to the store to the world,\nthen thand me to the sun a see to my see,\nso the sun a self all tooked thee,\nand do summer's dead with thy self thy self,\ni death the time that thou art thou forged,\nmy self in thee as a self all the sun,\ncomment to the world i have strange stole,\nwhich is the sun a shore to the time despise,\nwhich i desprice of the time that thou shoughtsome doth state the sun a see thee,\n"

So that is a good starting point, but we could do better. Lets first see how the number of LSTM cells affects the overall performance. We will train 100, 200 and 150 cell LSTMs, and then see if different optimizers yield different results.

In [209]:
# define model
model1 = Sequential()

model1.add(LSTM(100, input_shape=(X[0].shape)))
model1.add(Dense(encoding_size, activation='softmax'))

print(model1.summary())

# compile model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model1.fit(X, y, epochs=50, verbose=1)

output1 = 'from fairest creatures we desire increas'

l = 0
while l < 10:
    new = get_next(output1[-40:])
    if(new == '\n'):
        l+=1
    output1+=new
print(output1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dense_8 (Dense)              (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
87359/87359 [==============================] - 137s 2ms/step - loss: 2.4379 - acc: 0.3018
Epoch 2/50
87359/87359 [==============================] - 117s 1ms/step - loss: 2.0778 - acc: 0.3883
Epoch 3/50
87359/87359 [==============================] - 117s 1ms/step - loss: 1.9392 - acc: 0.4202
Epoch 4/50
87359/87359 [==============================] - 122s 1ms/step - loss: 1.8498 - acc: 0.4465
Epoch 5/50
87359/87359 [==============================] - 118s 1ms/step - loss: 1.7913 - acc: 0.4618
Epoch 6/50
87359/87359 [

So I tried to run this overnight, but my computer did not want to. Started again from here, reducing the epochs to 10.

In [211]:
# define model
model2 = Sequential()

model2.add(LSTM(200, input_shape=(X[0].shape)))
model2.add(Dense(encoding_size, activation='softmax'))

print(model2.summary())

# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model2.fit(X, y, epochs=10, verbose=1)

output2 = 'from fairest creatures we desire increas'

l = 0
while l < 10:
    new = get_next(output2[-40:])
    if(new == '\n'):
        l+=1
    output2+=new
print(output2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 200)               191200    
_________________________________________________________________
dense_10 (Dense)             (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
87359/87359 [==============================] - 173s 2ms/step - loss: 2.2801 - acc: 0.3404
Epoch 2/10
87359/87359 [==============================] - 182s 2ms/step - loss: 1.8673 - acc: 0.4414
Epoch 3/10
87359/87359 [==============================] - 186s 2ms/step - loss: 1.7194 - acc: 0.4768
Epoch 4/10
87359/87359 [==============================] - 188s 2ms/step - loss: 1.6204 - acc: 0.5028
Epoch 5/10
87359/87359 [==============================] - 185s 2ms/step - loss: 1.5418 - acc: 0.5244
Epoch 6/10
87359/87359

In [212]:
# define model
model3 = Sequential()

model3.add(LSTM(150, input_shape=(X[0].shape)))
model3.add(Dense(encoding_size, activation='softmax'))

print(model3.summary())

# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model3.fit(X, y, epochs=10, verbose=1)

output3 = 'from fairest creatures we desire increas'

l = 0
while l < 10:
    new = get_next(output3[-40:])
    if(new == '\n'):
        l+=1
    output3+=new
print(output3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 150)               113400    
_________________________________________________________________
dense_11 (Dense)             (None, 38)                5738      
Total params: 119,138
Trainable params: 119,138
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
87359/87359 [==============================] - 148s 2ms/step - loss: 2.3717 - acc: 0.3219
Epoch 2/10
87359/87359 [==============================] - 143s 2ms/step - loss: 1.9772 - acc: 0.4099
Epoch 3/10
87359/87359 [==============================] - 146s 2ms/step - loss: 1.8189 - acc: 0.4550
Epoch 4/10
87359/87359 [==============================] - 150s 2ms/step - loss: 1.7234 - acc: 0.4788
Epoch 5/10
87359/87359 [==============================] - 150s 2ms/step - loss: 1.6532 - acc: 0.4957
Epoch 6/10
87359/87359

I realized here that the poems I was outputting were based on the original model. The code below was modified to generate poems using the relevant model.

In [221]:
# define model
model4 = Sequential()

model4.add(LSTM(100, input_shape=(X[0].shape)))
model4.add(Dense(encoding_size, activation='softmax'))

print(model4.summary())

# compile model
model4.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

# fit model
model4.fit(X, y, epochs=10, verbose=1)

p4_1 = gen_poem(model4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dense_14 (Dense)             (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
87359/87359 [==============================] - 137s 2ms/step - loss: 2.4507 - acc: 0.2992
Epoch 2/10
87359/87359 [==============================] - 121s 1ms/step - loss: 2.0729 - acc: 0.3914
Epoch 3/10
87359/87359 [==============================] - 117s 1ms/step - loss: 1.9279 - acc: 0.4300
Epoch 4/10
87359/87359 [==============================] - 118s 1ms/step - loss: 1.8392 - acc: 0.4526
Epoch 5/10
87359/87359 [==============================] - 119s 1ms/step - loss: 1.7779 - acc: 0.4673
Epoch 6/10
87359/87359 [

In [222]:
# define model
model5 = Sequential()

model5.add(LSTM(100, input_shape=(X[0].shape)))
model5.add(Dense(encoding_size, activation='softmax'))

print(model5.summary())

# compile model
model5.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# fit model
model5.fit(X, y, epochs=10, verbose=1)

p5_1 = gen_poem(model4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dense_15 (Dense)             (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
87359/87359 [==============================] - 113s 1ms/step - loss: 2.3943 - acc: 0.3146
Epoch 2/10
87359/87359 [==============================] - 111s 1ms/step - loss: 1.9885 - acc: 0.4137
Epoch 3/10
87359/87359 [==============================] - 112s 1ms/step - loss: 1.8509 - acc: 0.4516
Epoch 4/10
87359/87359 [==============================] - 112s 1ms/step - loss: 1.7680 - acc: 0.4732
Epoch 5/10
87359/87359 [==============================] - 112s 1ms/step - loss: 1.7068 - acc: 0.4893
Epoch 6/10
87359/87359 [

In [228]:
print('\n Model 1')
p1_1 = gen_poem(model1)
print('\n Model 2')
p2_1 = gen_poem(model2)
print('\n Model 3')
p3_1 = gen_poem(model3)
print('\n Model 4')
p4_1 = gen_poem(model4)
print('\n Model 5')
p5_1 = gen_poem(model5)


 Model 1
from fairest creatures we desire increased
biuth po that thou, or tells not reteated,
but of losous suffeit of thy dead, or true deneess,
and view by ser it by night, or new truth,
and of their motting sive whether rends ancquence:
make love that thou full outamining thy macced,
it should confounding to be reliving,
and praise they did vances cunk their baro's sight,
i canf erityling wrace gives and grached
than hid love thrate the old a soeft remiew.


 Model 2
from fairest creatures we desire increase,
nor state with sun, in your line of his where.
in my feering should to diew and butt.
and sut that beauty pright the flame you gree,
of thy tomminess to chose onr thing eorer.
un all-every, bright thy boutts hath her blessed,
yet your proud might vade had auduse one,
your foce thou my should and kenters wored.
your proud life's i thou art fearst,
but fleating of more tall by antower's eye,


 Model 3
from fairest creatures we desire increase
thou laiked will stord to self abs

Overall, the 200 cell LSTM looks best, based on the number of epochs run. It looks almost equivalent to model 1, with only 1/5th of the epochs. However, lets try again with a deterministic approach.

In [229]:
print('\n Model 1')
p1_2 = gen_det_poem(model1)
print('\n Model 2')
p2_2 = gen_det_poem(model2)
print('\n Model 3')
p3_2 = gen_det_poem(model3)
print('\n Model 4')
p4_2 = gen_det_poem(model4)
print('\n Model 5')
p5_2 = gen_det_poem(model5)


 Model 1
from fairest creatures we desire increase,
but thou art but find the self thou arting,
and the cure a coutit hast i see the deeds.
then summer's filler so fair the world i will grow,
that i am all the true dear state eyes still,
thou wilt thou art thou art but there i chane
or all the world be the true dear prove,
to show i courses of the true dear heart
to make me but what i will be thou art still,
to show i that i am all the black age,


 Model 2
from fairest creatures we desire increase,
the sure with thy self thou wilt love thee,
and therefore that which i have spend, where you dost strievis,
the world the sweet seeming that with thee,
and therefore that which i have spend, where you dost strievis,
the world the sweet seeming that with thee,
and therefore that which i have spend, where you dost strievis,
the world the sweet seeming that with thee,
and therefore that which i have spend, where you dost strievis,
the world the sweet seeming that with thee,


 Model 3
from fa

It looks like deterministic models require more training to generate non-repeating poems. We will generate a few more poems with models 2 and 3, just to compare a little more.

In [230]:
print('\n Model 2')
p2_3 = gen_poem(model2)
p2_4 = gen_poem(model2)
p2_5 = gen_poem(model2)
print('\n Model 3')
p3_3 = gen_poem(model3)
p3_4 = gen_poem(model3)
p3_5 = gen_poem(model3)


 Model 2
from fairest creatures we desire increased.
thou art argament of thine ever tell.
for far well that beauty my lov's mace curis, where all know
whe haven my score thus sweet spy mine eyes,
and new to she in and so, thine one flest,
and praise made oden of excuse with thine enguch.
for night my finder, (ald muster from the ftime
which spoance art night shall gives alant.
un, mad thou shouldst for mourd, by addity,
which lies as heart it sade) you htan no, need,

from fairest creatures we desire increasake:
come fort youb, your false orn thee sabs coousence
the gaudy will loving bring deep summ,
arove that vory are pastirg strands berought:
bur vide i with thy sack and forbest teeds.
there, out thy sweet words mor tyrmect gonged,
duess not to grow) and he thought, without reseather spont.
and which i steal hoth, from thy nurge disgrace:
which tanthy sid, frack why some inater bast)
showing thy friends with murion ender:

from fairest creatures we desire increase,
why minter than

So these are proving too difficult to compare. I will go with the 2nd model, simply because it reached a higher training accuracy (0.61 vs 0.56). We will train the same model for an extra 15 epochs, see how it is doing, and then train for another 25 if needed.

In [231]:
# fit model
model2.fit(X, y, epochs=15, verbose=1)

p2_6 = gen_poem(model2)

Epoch 1/15
87359/87359 [==============================] - 168s 2ms/step - loss: 1.1755 - acc: 0.6279
Epoch 2/15
87359/87359 [==============================] - 171s 2ms/step - loss: 1.1223 - acc: 0.6437
Epoch 3/15
87359/87359 [==============================] - 171s 2ms/step - loss: 1.0742 - acc: 0.6579
Epoch 4/15
87359/87359 [==============================] - 175s 2ms/step - loss: 1.0298 - acc: 0.6715
Epoch 5/15
87359/87359 [==============================] - 155s 2ms/step - loss: 0.9899 - acc: 0.6836
Epoch 6/15
87359/87359 [==============================] - 150s 2ms/step - loss: 0.9511 - acc: 0.6955
Epoch 7/15
87359/87359 [==============================] - 151s 2ms/step - loss: 0.9177 - acc: 0.7061
Epoch 8/15
87359/87359 [==============================] - 151s 2ms/step - loss: 0.8898 - acc: 0.7153
Epoch 9/15
87359/87359 [==============================] - 151s 2ms/step - loss: 0.8621 - acc: 0.7235
Epoch 10/15
87359/87359 [==============================] - 150s 2ms/step - loss: 0.8375 - a

In [234]:
p2_7 = gen_det_poem(model2)

from fairest creatures we desire increasure,
they do but sweets with sweets, and my desire,
her from their preser assentle surmeand,
that i am sor thee, thy worth the grous and made,
but for his scythe come him on thee but bear,
my love is ansed do none eyes,
fair from her false-bandance is allowed,
but yet my rebeliv's bentomed, are breeds,
and nothing stands but in the state,
when i against my self it donds,



Both probabilistic and deterministic look good, but more training would not hurt. I'm getting lunch now, so I'll train it for another 25, which should take 1:15.

In [236]:
# fit model
model2.fit(X, y, epochs=25, verbose=1)

Epoch 1/25
87359/87359 [==============================] - 172s 2ms/step - loss: 0.7320 - acc: 0.7611
Epoch 2/25
87359/87359 [==============================] - 196s 2ms/step - loss: 0.7174 - acc: 0.7662
Epoch 3/25
87359/87359 [==============================] - 184s 2ms/step - loss: 0.7016 - acc: 0.7704 0s - loss: 0.7014 - ac
Epoch 4/25
87359/87359 [==============================] - 183s 2ms/step - loss: 0.6929 - acc: 0.7732
Epoch 5/25
87359/87359 [==============================] - 154s 2ms/step - loss: 0.6780 - acc: 0.7786
Epoch 6/25
87359/87359 [==============================] - 150s 2ms/step - loss: 0.6665 - acc: 0.7805
Epoch 7/25
87359/87359 [==============================] - 152s 2ms/step - loss: 0.6569 - acc: 0.7823
Epoch 8/25
87359/87359 [==============================] - 172s 2ms/step - loss: 0.6476 - acc: 0.7858
Epoch 9/25
87359/87359 [==============================] - 179s 2ms/step - loss: 0.6409 - acc: 0.7882
Epoch 10/25
87359/87359 [==============================] - 158s 2ms/

In [239]:
print('\n Probabilistic\n')
p2_8 = gen_poem(model2)

print('\n Deterministic\n')
p2_9 = gen_det_poem(model2)


 Probabilistic

from fairest creatures we desire increase,
thy praise twands in acquest, nor wron's:
this to make the hang mercards to time's ears him steed,
but be from thee, but sweet, doth note.
nor the fairest ond did neger tooned,
nothing thee, thy purpess are tur love doth be,
and slay they mourned boastaft be to gole,
for beauty's cold, with benory days are seens.
thine own not, prowers his guinted love
calms their powast doth life on them,
and place may gives thee, the better perce:
and al all men'st in thee of my fair frow,
cowarding with your recoure within decay,
resemblance of the dare point of sight,


 Deterministic

from fairest creatures we desire increase,
that life no longer than which makes art of me,
ham for my self i'll onned to are me.
such his sigulest shall fair great new sumb,
and that as thou alone, make the world must sing:
making no such outward past pleast disprained,
so doth the grous alone hours and proven,
the fair be new false i death doth live,
and do

That worked pretty well! Lets try increasing the window size to capture more than one line. That should theoretically yield better results, since we are covering more of the training data.

In [240]:
inputs2 = list()
input2_size = 60

for poem in poems:
    i = input2_size
    while i < len(poem)-1:
        inputs2.append([poem[i-input2_size:i], poem[i]])
        i+=1
        
X2 = list()
y2 = list()

for i in inputs2:
    X2.append(batch_encode(i[0]))
    y2.append(encode(i[1]))

X2 = np.array(X2)
y2 = np.array(y2)


In [241]:
X2.shape

(84279, 60, 38)

In [242]:
# define model
model6 = Sequential()

model6.add(LSTM(200, input_shape=(X2[0].shape)))
model6.add(Dense(encoding_size, activation='softmax'))

print(model6.summary())

# compile model
model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model6.fit(X2, y2, epochs=20, verbose=1)

p6_1 = gen_poem(model6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 200)               191200    
_________________________________________________________________
dense_16 (Dense)             (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
84279/84279 [==============================] - 268s 3ms/step - loss: 2.2917 - acc: 0.3354
Epoch 2/20
84279/84279 [==============================] - 252s 3ms/step - loss: 1.8845 - acc: 0.4353
Epoch 3/20
84279/84279 [==============================] - 211s 3ms/step - loss: 1.7284 - acc: 0.4741
Epoch 4/20
84279/84279 [==============================] - 213s 3ms/step - loss: 1.6299 - acc: 0.4996
Epoch 5/20
84279/84279 [==============================] - 213s 3ms/step - loss: 1.5502 - acc: 0.5206
Epoch 6/20
84279/84279

ValueError: Error when checking input: expected lstm_23_input to have shape (60, 38) but got array with shape (40, 38)

We also had to update the gen_poem function.

In [248]:
p6_1 = gen_poem(model6, poems[0][:60])

from fairest creatures we desire increase,
that thereby beauty should dead rehind enerhed,
all fair no fair with thine eye,
and weeens so strange, but shall, with fliend in every:
all thine all men after that herself imported,
i know the boint, and i strate's behime,
that in that fire me bestyest thou me, doth lide,
when athich to pracces if your love was will,
i patern in such riget discance of ertlime.
i worno on these what i worthle doth spent,
shall kind of meato's excenseans of thine eyes,
my lime disgrars now wrak the fone, and consert,
for if you not bemember thy self ine'en.
for blunten to their abusesty care of more:
shall other with this, and dead loved with shode,



In [249]:
p6_2 = gen_poem(model6, poems[0][:60])
p6_3 = gen_poem(model6, poems[0][:60])
p6_4 = gen_poem(model6, poems[0][:60])
p6_5 = gen_poem(model6, poems[0][:60])

from fairest creatures we desire increase,
that thereby beauty's fair throught to trought,
dill me strangelal inam in every slow,
that i am not love's frescised was still,
to ervers seem withil thou know strange astanted:
so lone but enjoy again was fears to end,
and pate which reeving this, riching subscove
tand thou my love more born morights bair,
and arten of every hadgen vowous,
as praise have unlaption to thy which ingrind.
hath'r thee give live thy newleth spitts and dead.
now bright despair thy lip, or well before,
but your beauty mad me hirv'st those tongue,
whose frimes in dark that never cure and dressing,
and to be a frisies fire more friled and write?

from fairest creatures we desire increase,
that thereby beauty, and thy faults consered,
without true lives on thy holous of your graces,
and strongly imporatize i day doth shows,
i love and arthory and the griefeled.
though moce mores, excause redoot hels with thee,
the incentummed leass me, since amised.
unly chire do this

However, the poems with a window size of 60 looked very good. If I did not speak english then you could absolutely convince me that these are good poems. This is mainly because it looks like english, even if it is not. Finally lets try decreasing the window size to 20.

In [250]:
inputs3 = list()
input3_size = 20

for poem in poems:
    i = input3_size
    while i < len(poem)-1:
        inputs3.append([poem[i-input3_size:i], poem[i]])
        i+=1
        
X3 = list()
y3 = list()

for i in inputs3:
    X3.append(batch_encode(i[0]))
    y3.append(encode(i[1]))

X3 = np.array(X3)
y3 = np.array(y3)


In [251]:
X3.shape

(90439, 20, 38)

In [254]:
# define model
model7 = Sequential()

model7.add(LSTM(200, input_shape=(X3[0].shape)))
model7.add(Dense(encoding_size, activation='softmax'))

print(model7.summary())

# compile model
model7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model7.fit(X3, y3, epochs=5, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 200)               191200    
_________________________________________________________________
dense_19 (Dense)             (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
90439/90439 [==============================] - 102s 1ms/step - loss: 2.2703 - acc: 0.3428
Epoch 2/5
90439/90439 [==============================] - 102s 1ms/step - loss: 1.8838 - acc: 0.4349
Epoch 3/5
90439/90439 [==============================] - 100s 1ms/step - loss: 1.7378 - acc: 0.4695
Epoch 4/5
90439/90439 [==============================] - 102s 1ms/step - loss: 1.6396 - acc: 0.4946
Epoch 5/5
90439/90439 [==============================] - 126s 1ms/step - loss: 1.5600 - acc: 0.5160


In [255]:
p7_1 = gen_poem(model7, poems[0][:20])

from fairest creature,
that spareth mish.
so dott datiand a jostens fortush of thy hast thou lov'st greanst post of prace,
keal doth woild wish a somening?
san love bald thing to the tim,
to be strengering owher, but the with can to praise,
beauteds freethous live your still out is his mest abuin:
nor that for the worth's with one.
that if shalt sick my gove,
wherey shink nosed so that a fore,
so muth recention not tone.
frilth notere sluble thee, thou white's ithen't trunged,
thon do alass as never,
and bearuy, a bestered doth with ander difthe here,



That did not work quite as well. This makes sense since we are covering less of the text, meaning there is less info to base future decisions off of. Lets try training a bit more, to see if we can increase the quality.

In [256]:
# fit model
model7.fit(X3, y3, epochs=5, verbose=1)

Epoch 1/5
90439/90439 [==============================] - 134s 1ms/step - loss: 1.4908 - acc: 0.5341
Epoch 2/5
90439/90439 [==============================] - 152s 2ms/step - loss: 1.4274 - acc: 0.5526
Epoch 3/5
90439/90439 [==============================] - 123s 1ms/step - loss: 1.3654 - acc: 0.5697
Epoch 4/5
90439/90439 [==============================] - 236s 3ms/step - loss: 1.3033 - acc: 0.5870
Epoch 5/5
90439/90439 [==============================] - 102s 1ms/step - loss: 1.2414 - acc: 0.6045


In [257]:
p7_2 = gen_poem(model7, poems[0][:20])

from fairest creatured deeds,
my viciles the piop in one, and you is do not tife,
the rate my soul, when of worth, with try speace mays what then winw.
o wo thy breath (d-live manss of i awoung thy beauty to say thy mimituen offect,
and sadce i astreatl,
hem and trues, or fur to ungrespe to ripery with pay a vering :
be,
but an your part be spown,
we to whete as thy vorst is suswer of your tootuqee near.
and all thin wornd me trilms a disfarxed things thy much of night
wand for my trues while wat a men,
in new he all my self tebbow,
which winturny beause of sweetcepget with subjectles in her writely of arthroses,
and you no'e must lecg, then 'ericted baded flowers,



We could not. In fact, the poem above actually looks worse than the first model 7 poem. Both are far overshadowed by model 6, which will most likely be the final one we go with.

In [258]:
p6_6 = gen_poem(model6, poems[0][:60])

from fairest creatures we desire increase,
that thereby beauty's songes' colfined beased,
but do not so long, and froshan and from thought
to wake the minded stain art so gurled lies,
lest soor with poor, and give make the rest?
no kif seems foul adour abtiquilainies
passoush one, ever so, no eat a tomber die,
thou canst to see as any, me fort so oft
as their is ton race, retths of sovereign pain.
as thought inlease him all divined pace weed,
thy proud ear kind, which to show wate in thee.
th' barths even as fot than wetersh store,
haw dupy praise i dide not in worth thy seek,
but you are your slame to make the sugh my migd.
hampies to spain to make to wook being mane.



In [276]:
model8 = Sequential()
model8.add(LSTM(250, input_shape=(X2[0].shape)))
model8.add(Dense(encoding_size, activation='softmax'))
           
model8.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model8.summary())

# fit model
model8.fit(X2, y2, epochs=2, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 250)               289000    
_________________________________________________________________
dense_24 (Dense)             (None, 38)                9538      
Total params: 298,538
Trainable params: 298,538
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
84279/84279 [==============================] - 349s 4ms/step - loss: 2.2972 - acc: 0.3349
Epoch 2/2
84279/84279 [==============================] - 317s 4ms/step - loss: 1.8837 - acc: 0.4332


We tried increasing the LSTM size, but it appears that 200 works best. As such, we will finish part 6 here. 